In [1]:
import pandas as pd 
import numpy as np 
import math
from datetime import datetime
from nsepython import *


In [2]:
def _get_option_variables(ticker:str):
    """args: This function take ticker in string format
       output: Option Chain Values
    """
    ticker = ticker.upper()
    ticker = 'INFY'

    # Get the option chain
    option_chain = nse_optionchain_scrapper(ticker)
    records = option_chain['records']['data']

    # Get current stock price
    S0 = option_chain['records']['underlyingValue']

    # Filter only calls (you can also choose puts)
    call_options = [item for item in records if 'CE' in item]

    # Pick the ATM (closest strike to S0)
    atm_option = min(call_options, key=lambda x: abs(x['CE']['strikePrice'] - S0))
    K = atm_option['CE']['strikePrice']
    market_price = atm_option['CE']['lastPrice']
    implied_volatility = atm_option['CE']['impliedVolatility']# decimal like 0.20
    expiry = atm_option['CE']['expiryDate']

    # Compute time to maturity in years
    expiry_date = datetime.datetime.strptime(expiry, '%d-%b-%Y')
    print("Market Price :",market_price)
    print("Implied Volatility :",implied_volatility)
    print("Expiry Date :",expiry)




In [ ]:
def Monte_Carlo_Simulator(N,option_price,strike_price,expiry_date,volatility,risk_free_rate,CE=True):
    simualtions = 0
    today = datetime.today()
    day_diff = (expiry_date - today).days
    num_of_steps = day_diff
    T = day_diff/252 ## number of trading days
    r = risk_free_rate
    dt = T/num_of_steps
    simualtions = np.zeros((num_of_steps,N))
    i = 1
    simualtions[0] = option_price
    while i<num_of_steps:
        i+=1
        Z = np.random.standard_normal(N)
        simualtions[i] = simualtions[i-1]*np.exp(r-0.5*volatility**2)*dt + volatility*np.sqrt(dt)*Z
    
    payoff_matrix = np.zeros((num_of_steps,N))
    if CE:
        payoff_matrix = max(simualtions-strike_price,0)
    else:
        payoff_matrix = max(strike_price-simualtions,0)
    
    return payoff_matrix




